In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
nltk.download() #Necessary to use this part of the code, DF should be exported afterwards so no need of this being used afterwards.
tweets = pd.read_csv('withoutEncoding.csv')

In [ ]:
s = "<pre>	1.	CC	Coordinating conjunction<br>	2.	CD	Cardinal number<br>	3.	DT	Determiner<br>	4.	EX	Existential <em>there</em><br>	5.	FW	Foreign word<br>	6.	IN	Preposition or subordinating conjunction<br>	7.	JJ	Adjective<br>	8.	JJR	Adjective, comparative<br>	9.	JJS	Adjective, superlative<br>	10.	LS	List item marker<br>	11.	MD	Modal<br>	12.	NN	Noun, singular or mass<br>	13.	NNS	Noun, plural<br>	14.	NNP	Proper noun, singular<br>	15.	NNPS	Proper noun, plural<br>	16.	PDT	Predeterminer<br>	17.	POS	Possessive ending<br>	18.	PRP	Personal pronoun<br>	19.	PRP$	Possessive pronoun<br>	20.	RB	Adverb<br>	21.	RBR	Adverb, comparative<br>	22.	RBS	Adverb, superlative<br>	23.	RP	Particle<br>	24.	SYM	Symbol<br>	25.	TO	<em>to</em><br>	26.	UH	Interjection<br>	27.	VB	Verb, base form<br>	28.	VBD	Verb, past tense<br>	29.	VBG	Verb, gerund or present participle<br>	30.	VBN	Verb, past participle<br>	31.	VBP	Verb, non-3rd person singular present<br>	32.	VBZ	Verb, 3rd person singular present<br>	33.	WDT	Wh-determiner<br>	34.	WP	Wh-pronoun<br>	35.	WP$	Possessive wh-pronoun<br>	36.	WRB	Wh-adverb<br></pre>"
#Extracted from https://cs.nyu.edu/grishman/jet/guide/PennPOS.html
tags = set(re.findall(r"\t(\w+)\t",s) + ['PRP$', 'WP$'])
def getTags(text):
    finalArray = []
    possibleTags = nltk.pos_tag(nltk.TweetTokenizer().tokenize(text))
    for word in possibleTags:
        if word[1] in tags:
            finalArray.append(word)
    return finalArray

In [ ]:
df = tweets['text'].apply(getTags)
df = df.reset_index()
df

In [ ]:
def _isInCriteria(textToAnalize, criteriaArray):
    for value in criteriaArray:
        if textToAnalize.startswith(value):
            return True
    return False

def keepOnlyCriteria(listWords, criteria = 'VB'):
    newArray = []
    for value in listWords:
        if _isInCriteria(value[1], [criteria]):
            newArray.append(value)
    return newArray

def excludeCriteria(listWords, criteria):
    newArray = []
    for value in listWords:
        if (_isInCriteria(value[1], criteria)):
            continue
        newArray.append(value)
    return newArray
        

In [ ]:
df['amountOfVerbs'] = df['text'].apply(keepOnlyCriteria).apply(len)
df['amountOfPronouns'] = df['text'].apply(lambda x: len(keepOnlyCriteria(x, 'PR') + keepOnlyCriteria(x, 'WP')))
df['amountOfNouns'] = df['text'].apply(lambda x: len(keepOnlyCriteria(x, 'NN')))
df['amountOfAdjectives'] = df['text'].apply(lambda x: len(keepOnlyCriteria(x, 'JJ')))
df['amountOfAdverbs'] = df['text'].apply(lambda x: len(keepOnlyCriteria(x, 'RB')))
df['restOfWords'] = df['text'].apply(lambda x: len(excludeCriteria(x, ['RB', 'JJ', 'NN', 'PR', 'WP', 'VB'])))
df['totalAmount'] = df['text'].apply(len)

In [ ]:
finalDf = df.drop(['index', 'text'], axis=1)
finalDf['id'] = tweets['id']
finalDf.to_csv('wordCount.csv', index = False) 